# Model 6: Kernel Initialization

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
batch_size = 8
n_classes = 20
n_epochs = 30
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1 - Glorot Normal Initialization

In [6]:
# defining the model architecture
m6_t1 = models.Sequential()

# convolution/max pool stacks
m6_t1.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same', kernel_initializer='glorot_normal'))
m6_t1.add(layers.LeakyReLU(alpha=0.1))
m6_t1.add(layers.MaxPooling2D((2,2)))

m6_t1.add(layers.Conv2D(64,(3,3), padding='same', kernel_initializer='glorot_normal'))
m6_t1.add(layers.LeakyReLU(alpha=0.1))
m6_t1.add(layers.MaxPooling2D((2,2)))

m6_t1.add(layers.Conv2D(128,(3,3), padding='same', kernel_initializer='glorot_normal'))
m6_t1.add(layers.LeakyReLU(alpha=0.1))
m6_t1.add(layers.MaxPooling2D((2,2)))

m6_t1.add(layers.Conv2D(256,(3,3), padding='same', kernel_initializer='glorot_normal'))
m6_t1.add(layers.LeakyReLU(alpha=0.1))
m6_t1.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m6_t1.add(layers.Flatten())
m6_t1.add(layers.Dense(512, kernel_initializer='glorot_normal'))
m6_t1.add(layers.LeakyReLU(alpha=0.1))
m6_t1.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m6_t1.summary()
model_names.append('m6_t1')
model_list.append(m6_t1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)      

### Trial 2 - Glorot Uniform Initialization (model_05 re-run)

In [7]:
# defining the model architecture
m6_t2 = models.Sequential()

# convolution/max pool stacks
m6_t2.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same', kernel_initializer='glorot_uniform'))
m6_t2.add(layers.LeakyReLU(alpha=0.1))
m6_t2.add(layers.MaxPooling2D((2,2)))

m6_t2.add(layers.Conv2D(64,(3,3), padding='same', kernel_initializer='glorot_uniform'))
m6_t2.add(layers.LeakyReLU(alpha=0.1))
m6_t2.add(layers.MaxPooling2D((2,2)))

m6_t2.add(layers.Conv2D(128,(3,3), padding='same', kernel_initializer='glorot_uniform'))
m6_t2.add(layers.LeakyReLU(alpha=0.1))
m6_t2.add(layers.MaxPooling2D((2,2)))

m6_t2.add(layers.Conv2D(256,(3,3), padding='same', kernel_initializer='glorot_uniform'))
m6_t2.add(layers.LeakyReLU(alpha=0.1))
m6_t2.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m6_t2.add(layers.Flatten())
m6_t2.add(layers.Dense(512, kernel_initializer='glorot_uniform'))
m6_t2.add(layers.LeakyReLU(alpha=0.1))
m6_t2.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m6_t2.summary()
model_names.append('m6_t2')
model_list.append(m6_t2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 128)      

### Trial 3 - He Normal Initialization

In [8]:
# defining the model architecture
m6_t3 = models.Sequential()

# convolution/max pool stacks
m6_t3.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same', kernel_initializer='he_normal'))
m6_t3.add(layers.LeakyReLU(alpha=0.1))
m6_t3.add(layers.MaxPooling2D((2,2)))

m6_t3.add(layers.Conv2D(64,(3,3), padding='same', kernel_initializer='he_normal'))
m6_t3.add(layers.LeakyReLU(alpha=0.1))
m6_t3.add(layers.MaxPooling2D((2,2)))

m6_t3.add(layers.Conv2D(128,(3,3), padding='same', kernel_initializer='he_normal'))
m6_t3.add(layers.LeakyReLU(alpha=0.1))
m6_t3.add(layers.MaxPooling2D((2,2)))

m6_t3.add(layers.Conv2D(256,(3,3), padding='same', kernel_initializer='he_normal'))
m6_t3.add(layers.LeakyReLU(alpha=0.1))
m6_t3.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m6_t3.add(layers.Flatten())
m6_t3.add(layers.Dense(512, kernel_initializer='he_normal'))
m6_t3.add(layers.LeakyReLU(alpha=0.1))
m6_t3.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m6_t3.summary()
model_names.append('m6_t3')
model_list.append(m6_t3)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 50, 50, 128)      

### Trial 4 - He Uniform Initialization

In [9]:
# defining the model architecture
m6_t4 = models.Sequential()

# convolution/max pool stacks
m6_t4.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same', kernel_initializer='he_uniform'))
m6_t4.add(layers.LeakyReLU(alpha=0.1))
m6_t4.add(layers.MaxPooling2D((2,2)))

m6_t4.add(layers.Conv2D(64,(3,3), padding='same', kernel_initializer='he_uniform'))
m6_t4.add(layers.LeakyReLU(alpha=0.1))
m6_t4.add(layers.MaxPooling2D((2,2)))

m6_t4.add(layers.Conv2D(128,(3,3), padding='same', kernel_initializer='he_uniform'))
m6_t4.add(layers.LeakyReLU(alpha=0.1))
m6_t4.add(layers.MaxPooling2D((2,2)))

m6_t4.add(layers.Conv2D(256,(3,3), padding='same', kernel_initializer='he_uniform'))
m6_t4.add(layers.LeakyReLU(alpha=0.1))
m6_t4.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m6_t4.add(layers.Flatten())
m6_t4.add(layers.Dense(512, kernel_initializer='he_uniform'))
m6_t4.add(layers.LeakyReLU(alpha=0.1))
m6_t4.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m6_t4.summary()
model_names.append('m6_t4')
model_list.append(m6_t4)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 50, 50, 128)      

## Step 4: Training

In [10]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
%%time
# data standardization and augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

CPU times: user 403 ms, sys: 861 ms, total: 1.26 s
Wall time: 1.27 s


In [12]:
# compiling loss functions
m6_t1.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])
m6_t2.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])
m6_t3.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])
m6_t4.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])

In [13]:
%%time
# setting up model saving checkpoints
m6_t1_cp = ModelCheckpoint(filepath=f'{model_path}m6_t1.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m6_t1_history = m6_t1.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m6_t1_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m6_t1_history)

Epoch 1/30
1013/1013 [==============================] - 77s 76ms/step - loss: 2.6710 - acc: 0.1541 - val_loss: 2.6768 - val_acc: 0.2822

Epoch 00001: val_acc improved from -inf to 0.28218, saving model to /floyd/home/m6_t1.h5
Epoch 2/30
1013/1013 [==============================] - 77s 76ms/step - loss: 2.3122 - acc: 0.2578 - val_loss: 1.9666 - val_acc: 0.3392

Epoch 00002: val_acc improved from 0.28218 to 0.33915, saving model to /floyd/home/m6_t1.h5
Epoch 3/30
1013/1013 [==============================] - 77s 76ms/step - loss: 2.1752 - acc: 0.2953 - val_loss: 2.5922 - val_acc: 0.4239

Epoch 00003: val_acc improved from 0.33915 to 0.42394, saving model to /floyd/home/m6_t1.h5
Epoch 4/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.0961 - acc: 0.3300 - val_loss: 2.1492 - val_acc: 0.4140

Epoch 00004: val_acc did not improve from 0.42394
Epoch 5/30
1013/1013 [==============================] - 77s 76ms/step - loss: 1.9991 - acc: 0.3516 - val_loss: 1.2973 - val_acc: 

In [14]:
%%time
# setting up model saving checkpoints
m6_t2_cp = ModelCheckpoint(filepath=f'{model_path}m6_t2.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m6_t2_history = m6_t2.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m6_t2_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m6_t2_history)

Epoch 1/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.7505 - acc: 0.1386 - val_loss: 2.9185 - val_acc: 0.2153

Epoch 00001: val_acc improved from -inf to 0.21535, saving model to /floyd/home/m6_t2.h5
Epoch 2/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.3673 - acc: 0.2264 - val_loss: 1.8793 - val_acc: 0.3092

Epoch 00002: val_acc improved from 0.21535 to 0.30923, saving model to /floyd/home/m6_t2.h5
Epoch 3/30
1013/1013 [==============================] - 77s 76ms/step - loss: 2.2187 - acc: 0.2831 - val_loss: 1.8935 - val_acc: 0.3516

Epoch 00003: val_acc improved from 0.30923 to 0.35162, saving model to /floyd/home/m6_t2.h5
Epoch 4/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.1186 - acc: 0.3216 - val_loss: 1.9212 - val_acc: 0.4302

Epoch 00004: val_acc improved from 0.35162 to 0.43017, saving model to /floyd/home/m6_t2.h5
Epoch 5/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.0216 - 

In [15]:
%%time
# setting up model saving checkpoints
m6_t3_cp = ModelCheckpoint(filepath=f'{model_path}m6_t3.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m6_t3_history = m6_t3.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m6_t3_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m6_t3_history)

Epoch 1/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.9541 - acc: 0.1677 - val_loss: 2.9807 - val_acc: 0.2723

Epoch 00001: val_acc improved from -inf to 0.27228, saving model to /floyd/home/m6_t3.h5
Epoch 2/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.3421 - acc: 0.2572 - val_loss: 2.1329 - val_acc: 0.3566

Epoch 00002: val_acc improved from 0.27228 to 0.35661, saving model to /floyd/home/m6_t3.h5
Epoch 3/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.2194 - acc: 0.2860 - val_loss: 2.4192 - val_acc: 0.3666

Epoch 00003: val_acc improved from 0.35661 to 0.36658, saving model to /floyd/home/m6_t3.h5
Epoch 4/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.1033 - acc: 0.3270 - val_loss: 1.7728 - val_acc: 0.4501

Epoch 00004: val_acc improved from 0.36658 to 0.45012, saving model to /floyd/home/m6_t3.h5
Epoch 5/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.0029 - 

In [16]:
%%time
# setting up model saving checkpoints
m6_t4_cp = ModelCheckpoint(filepath=f'{model_path}m6_t4.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m6_t4_history = m6_t4.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m6_t4_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m6_t4_history)

Epoch 1/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.9465 - acc: 0.1719 - val_loss: 2.0469 - val_acc: 0.2995

Epoch 00001: val_acc improved from -inf to 0.29950, saving model to /floyd/home/m6_t4.h5
Epoch 2/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.3500 - acc: 0.2557 - val_loss: 1.6644 - val_acc: 0.3229

Epoch 00002: val_acc improved from 0.29950 to 0.32294, saving model to /floyd/home/m6_t4.h5
Epoch 3/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.2191 - acc: 0.2881 - val_loss: 1.2348 - val_acc: 0.3429

Epoch 00003: val_acc improved from 0.32294 to 0.34289, saving model to /floyd/home/m6_t4.h5
Epoch 4/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.0843 - acc: 0.3346 - val_loss: 2.6100 - val_acc: 0.3953

Epoch 00004: val_acc improved from 0.34289 to 0.39526, saving model to /floyd/home/m6_t4.h5
Epoch 5/30
1013/1013 [==============================] - 76s 75ms/step - loss: 2.0163 - 

In [17]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [18]:
# evaluating models on validation set
for key, value in models_dict.items():
    model = models.load_model(f'{model_path}{key}.h5')
    (val_loss, val_accuracy) = model.evaluate(val_generator,verbose=1)
    print(f'{key} Val Accuracy: {round((val_accuracy*100),2)}%')
    print(f'{key} Val Loss: {round(val_loss,4)}')
    print('---')

102/102 [==============================] - 2s 17ms/step
m6_t1 Val Accuracy: 64.57%
m6_t1 Val Loss: 1.0852
---
102/102 [==============================] - 2s 17ms/step
m6_t2 Val Accuracy: 66.67%
m6_t2 Val Loss: 2.523
---
102/102 [==============================] - 2s 17ms/step
m6_t3 Val Accuracy: 63.46%
m6_t3 Val Loss: 1.7824
---
102/102 [==============================] - 2s 17ms/step
m6_t4 Val Accuracy: 65.31%
m6_t4 Val Loss: 1.911
---


In [19]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

m6_t1_history saved in /floyd/home/
m6_t2_history saved in /floyd/home/
m6_t3_history saved in /floyd/home/
m6_t4_history saved in /floyd/home/
